# Tensorflow Tutorial

## 1. Graph

In [3]:
import tensorflow as tf 
import numpy as np

간단하게 그래프를 생성해서 tensorboard를 통해 살펴보기로 하자. <br/>
현재 working directory에 ./log에 로그를 저장하도록 한다.

현재 그래프를 반환한다.
```
tf.get_default_graph()
```

In [55]:
tf.reset_default_graph()

a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)

total = a + b

# tensorboard를 위해 그래프 추가
# 그래프가 완성된 뒤 추가 되어야 한다.
writer = tf.summary.FileWriter('./log', tf.get_default_graph())

with tf.Session() as sess:
    print(sess.run({'ab':(a, b), 'total':total}), '\n')
    
# print(tf.get_default_graph().as_graph_def())    

{'ab': (3.0, 4.0), 'total': 7.0} 



### 1.1 Tensorboard

포트를 특별히 지정하지 않으면, 디폴트 포트는 6006이다.<br/>
tensorboard의 실행 방법은 다음과 같다.

```
$ tensorboard --logdir=<path> --port=6006
```

위 예제 실행은 다음과 같다.

```
$ tensorboard --logdir=./log --port=6006
```

그래프의 동작 방식을 이해하기 위한 예제이다.<br/>
다음은 그래프가 3번 실행 된다.<br/>
마지막 실행에서 out1, out2가 동일한 vec값을 사용했다는 점을 확인하기 바란다.

In [56]:
tf.reset_default_graph()

#변수 3개 반환
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2

with tf.Session() as sess:
    print('vec: {}'.format(sess.run(vec)))
    print('vec: {}'.format(sess.run(vec)))
    # out1, out2가 동일한 vec을 사용했다
    print('out: {}'.format(sess.run((out1, out2))))

vec: [0.8685545  0.15405309 0.16789365]
vec: [0.95555186 0.9513595  0.7278329 ]
out: (array([1.3245795, 1.9598197, 1.9704139], dtype=float32), array([2.3245795, 2.9598198, 2.970414 ], dtype=float32))


### 1.2 Constants

그래프에 임의의 상수값을 지정하여 추가할 수 있다.

```python
tf.constant(
    value,
    dtype=None,
    shape=None,
    name='Const',
    verify_shape=False
)
```

지정된 상수는 그래프와 함께 저장된다.<br/>
다음과 같이 값을 지정해서 텐서를 생성할 수 있다.<br/>
첫번째 파라미터가 shape이 아닌 값이다.

In [11]:
tf.reset_default_graph()

# constant of 0d tensor (scalar)
a = tf.constant(2, name='scalar')
# constant of 1d tensor (vector)
b = tf.constant([2, 2], name='vector')
# constant of 2x2 tensor (matrix)
c = tf.constant([[0,1], [2, 3]], name='matrix')

with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))

()
(2,)
(2, 2)
2
[2 2]
[[0 1]
 [2 3]]


constant외에도 다양한 상수 관련 함수가 있으니 API 문서에서 확인하기 바란다.

In [57]:
tf.reset_default_graph()

a = tf.zeros([2, 3])
b = tf.zeros_like(a)
c = tf.ones([2, 3])
d = tf.ones_like(c)
e = tf.fill([2,3], 8)
f = tf.lin_space(10.0, 13.0, 4, name='linspace')
start = 3
limit = 18
delta = 3
g = tf.range(start, limit, delta)
h = tf.range(limit)

with tf.Session() as sess:
    print('a', sess.run(a))
    print('b', sess.run(b))
    print('c', sess.run(c))
    print('d', sess.run(d))
    print('e', sess.run(e))
    print('f', sess.run(f))
    print('g', sess.run(g))
    print('h', sess.run(h))

a [[0. 0. 0.]
 [0. 0. 0.]]
b [[0. 0. 0.]
 [0. 0. 0.]]
c [[1. 1. 1.]
 [1. 1. 1.]]
d [[1. 1. 1.]
 [1. 1. 1.]]
e [[8 8 8]
 [8 8 8]]
f [10. 11. 12. 13.]
g [ 3  6  9 12 15]
h [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]


### 1.3 Variables

* 앞에서 살펴본 tf.constant는 함수이다. 하지만 tf.Variable은 여러게의 함수로 구성된 클래스다.
* tf.Variable 쓰지 말고 tf.get_variable을 사용하도록 한다.

```python
tf.get_variable(
    name,
    shape=None,
    dtype=None,
    initializer=None,
    regularizer=None,
    trainable=True,
    collections=None,
    caching_device=None,
    partitioner=None,
    validate_shape=True,
    use_resource=None,
    custom_getter=None,
    constraint=None
)
```

* tf.constant는 그래프에 상수화되어 저장되지만 Variable은 parameter server에 메모리 상에 저장된다.
* Variable은 사용하기 전 초기화가 선행되어야 한다.

* 다음 함수를 호출하면 지정된 initializer가 호출된다.
```
tf.global_variables_initializer()
```
* 그 밖에 Variable을 개별로 초기화를 해주는 방법과 assign을 이용하는 방법은 생략하겠다. 

In [27]:
# 같은 이름의 variable을 그래프에 추가할 수 없기 때문에 그래프를 리셋해줘야 한다.
tf.reset_default_graph()

v = tf.get_variable(name='normal_matrix', shape=(3,3), initializer=tf.truncated_normal_initializer())

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(v))

[[ 0.5987738  -1.3011031  -0.2660086 ]
 [ 1.633186    0.15406503 -0.8655053 ]
 [-1.1830344  -1.8143717   1.269143  ]]


### 1.4 Placeholders

데이터를 그래프에 임포트하기 위해서는 placeholder를 지정해 줘야한다.

```python
tf.placeholder(
    dtype,
    shape=None,
    name=None
)
```

* 과거에 사용하던 feed_dict를 이용해서 입력 값을 지정하던 방식은 지양하는 것이 좋다.
* 뒤에 설명할 tf.data.Dataset을 이용하는 방법이 좋다.
* placeholder의 shape 파라미터를 None으로 지정해도 되지면, 권장되지는 않으니 꼭 지정해 주는 것이 좋다.

다음과 같이 간단히 살펴보기로 하자

In [58]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

z = x + y
with tf.Session() as sess:
    print(sess.run(z, feed_dict={x: 3, y: 4.5}))
    print(sess.run(z, feed_dict={x: [1, 3], y:[2, 4]}))

7.5
[3. 7.]


### 1.5 feed_dict을 이용한 디버깅 기법
placeholder 뿐만 아니라 constant, variable 모두 feedable object로 임의의 값을 지정하여 수식을 확인할 수 있다. <br/>
해당 그래프가 정상적으로 동작하는지 확인할 때, 아주 유용하니 꼭 알아두기 바란다.

In [59]:
import tensorflow as tf

tf.reset_default_graph()

batch_size = 3
features = 2
n_hiden = 3

x = tf.placeholder(dtype=tf.float32, shape=[batch_size, features], name='batch_input')

w = tf.get_variable(name='weights', dtype=tf.float32, shape=(features, n_hiden))
b = tf.get_variable(name='bias', dtype=tf.float32, shape=(1, n_hiden))

m = tf.matmul(x, w)
result = tf.add(m, b)

tf.summary.FileWriter('./log', tf.get_default_graph())

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    feed_dict = {
        x: [[1, 2],
            [3, 4],
            [5, 6]],

        w: [[1, 2, 3],
            [3, 4, 5]],

        b: [[10, 10, 10]]
    }

    print(sess.run([m], feed_dict=feed_dict))

[array([[ 7., 10., 13.],
       [15., 22., 29.],
       [23., 34., 45.]], dtype=float32)]


### 1.6 Control flow

tensorflow는 Graph에 조건에 따른 분기를 추가하기 위해서 다양한 함수를 제공한다. <br/>
여기서는 기본이 되는 tf.cond를 살펴보기로 하겠다.

```python
tf.cond(
    pred,
    true_fn=None,
    false_fn=None,
    strict=False,
    name=None,
    fn1=None,
    fn2=None
)
```

예제는 Huber loss를 사용하겠다. 상세한 내용은 [링크](https://en.wikipedia.org/wiki/Huber_loss)를 참고하기 바란다.<br/>
Huber loss function은 다음과 같다.

![alt Huberloss](img/huberloss.svg)

델타 값을 기준으로 Loss값이 다르게 생성된다.


In [54]:
tf.reset_default_graph()

def huber_loss(label, prediction, delta):
    residual = tf.abs(label - prediction)
    def f1(): return 0.5 * tf.square(residual)
    def f2(): return delta * residual - 0.5 * tf.square(delta)
    return tf.cond(tf.less(residual, delta), f1, f2)

label = tf.placeholder(name='lable', shape=[], dtype=tf.float32)
prediction = tf.placeholder(name='prediction', shape=[], dtype=tf.float32)

loss = huber_loss(label, prediction, 4.0)

with tf.Session() as sess:
    print(sess.run(loss, feed_dict={ label: 1, prediction: 3})) # 0.5 * 2.0 * 2.0
    print(sess.run(loss, feed_dict={ label: 1, prediction: 11})) # (4.0 * 10) - (0.5 * 4.0 * 4.0)

2.0
32.0


## 2 Dataset

딥러닝으로 시스템을 구성하거나 다양한 학습데이터 학습하기 위해서 입력 데이터 구성은 아주 중요하다.<br/>
때로는 모델을 디자인하고 만드는 것보다 데이터 파이프 라인을 구성하는데 더 오랜 시간이 걸리기도 한다. <br/>
외부에서 데이터를 가공하여 placeholder에 feed_dict를 이용해서 직접 데이터를 넣어 주는 방식을 지양하는 것이 좋다.<br/>
tf.data.Dataset을 반드시 익혀서 사용하도록 하도록 하자

```python
@staticmethod
from_tensor_slices(tensors)
```

```python
@staticmethod
from_generator(
    generator,
    output_types,
    output_shapes=None,
    args=None
)
```


In [ ]:
source = np.array([
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7],
])

slices = tf.data.Dataset.from_tensor_slices(source)
# 초기화 필요 없다.
iterator = slices.make_one_shot_iterator()
next_item = iterator.get_next()

with tf.Session() as sess:
    idx = 0
    while True:
        try:
            idx += 1
            print(f'{idx}: ',sess.run(next_item))
        except tf.errors.OutOfRangeError:
            break


In [ ]:
source = tf.random_normal(shape=[10, 3])
slices = tf.data.Dataset.from_tensor_slices(source)
# 초기화 필요. (뒤에서 추가 설명)
iterator = slices.make_initializable_iterator()
next_row = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer)
    
    i = 0
    while True:
        try:
            i+=1
            print(i, sess.run(next_row))
        except tf.errors.OutOfRangeError:
            break

In [ ]:
source = tf.random_uniform([4, 10], minval=0, maxval=100)
# 입력 소스로 부터 dataset을 구성
dataset1 = tf.data.Dataset.from_tensor_slices(source)

print(dataset1.output_types)
print(dataset1.output_shapes)

inputs = tf.random_uniform([4, 100])
labels = tf.random_uniform([4])

# 다른 입력 소스를 묶어서 dataset을 구성
dataset2 = tf.data.Dataset.from_tensor_slices((inputs, labels))

print(dataset2.output_types)
print(dataset2.output_shapes)

# dataset을 묶어서 새로운 dataset을 구성
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

print(dataset3.output_types)
print(dataset3.output_shapes)


In [ ]:
# 딕셔너리를 이용한 입력소스 지정 방법. key에 이름을 부여할 수 있음
sources = dict()

sources['a'] = tf.random_uniform([4])
sources['b'] = tf.random_uniform([4, 100])

dataset = tf.data.Dataset.from_tensor_slices(sources)

print(dataset.output_types)
print(dataset.output_shapes)


### 2.1 Iterator
iterator 사용 시,range 체크를 반드시 해주는 것이 좋다.

```python
try:
    do something
except tf.errors.OutOfRangeError:
    break
```

#### 2.1.1 one-shot iterator
* initialization이 필요 없다. (할 수 없다)
* 끝에 도착하면 다시 재활용할 수 없다. 결국, epoch마다 사용 불가능.

In [ ]:
dataset = tf.data.Dataset.range(10)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    while True:
        try:  
          print(sess.run(next_element))
        except tf.errors.OutOfRangeError:
            break

#### 2.1.2 initalizable iterator
* 사용전, initialization 필수
* epoch마다 재 활용할 수 있다.
* tf.placeholder를 이용해서 동작을 지정할 있는 장점이 있다.

In [ ]:
max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(max_value)

iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:

    for epoch in range(2):
        sess.run(iterator.initializer, feed_dict={max_value: 20})
        while True:
            try:
                print(f'epoch {epoch}: ', sess.run(next_element))
            except tf.errors.OutOfRangeError:
                break
            
        print('\n')

#### 2.1.3 reinitilizable iterator
* 여러 Data source로 부터 데이터를 가지고 올수 있다. 단, structure은 동일해야 한다.

In [ ]:
training_dataset = tf.data.Dataset.range(10).map(lambda x: x + tf.random_uniform([], -10, 10, tf.int64))
validation_dataset = tf.data.Dataset.range(5)

# reinitializable iterator의 경우는 structure로 정의된다.
# 다른 Data source라 할지라도 같은 데이터 형태를 가지기 때문에 둘 중 하나를 사용하면 된다.
iterator = tf.data.Iterator.from_structure(training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()

training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

with tf.Session() as sess:
    
    #The number of epochs
    for e in range(20):
        print(f'> epoch {e}')
        
        # train sequence가 초기화 된다.
        sess.run(training_init_op)
        for _ in range(10):
            print(f'train: {sess.run(next_element)}')
        # validation sequence가 초기화 된다.
        sess.run(validation_init_op)
        for _ in range(5):
            print(f'valid: {sess.run(next_element)}')

#### 2.1.4 feedable iterator
* reinitializable iterator와 동일 기능 제공

In [ ]:
# repeat함수를 이용해서 반복되는 sequence를 생성할 수 있다.
# repeat함수를 사용하지 않으면 범위를 벗어날 경우, OutOfRangeError가 발생한다.
training_dataset = tf.data.Dataset.range(10).repeat()
validation_dataset = tf.data.Dataset.range(5)

handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(handle, 
                                               validation_dataset.output_types, 
                                               validation_dataset.output_shapes)

next_element = iterator.get_next()

training_iterator = training_dataset.make_one_shot_iterator()
validation_iterator = validation_dataset.make_initializable_iterator()


with tf.Session() as sess:
    # input 소스를 선택할 수 있는 핸들러
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(validation_iterator.string_handle())
    
    while True:
        for _ in range(200):
            print('train ', sess.run(next_element, feed_dict={handle: training_handle}))

        sess.run(validation_iterator.initializer)
        for _ in range(5):
            print('valid ', sess.run(next_element, feed_dict={handle: validation_handle}))

In [ ]:
dataset = tf.data.Dataset.range(3)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()
result = tf.add(next_element, next_element)

with tf.Session() as sess:
    sess.run(iterator.initializer)
   
    # 일반적으로 train loop에 사용하는 try-except 블록
    try:
        print(sess.run(result)) # 0
        print(sess.run(result)) # 1
        print(sess.run(result)) # 2
    except tf.errors.OutOfRangeError:
        print('End of dataset')

In [ ]:
dataset1 = tf.data.Dataset.from_tensor_slices(['A', 'B', 'C'])
dataset2 = tf.data.Dataset.from_tensor_slices(([1, 2, 3], ['A1', 'B1', 'C1']))
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

iterator = dataset3.make_initializable_iterator()
next1, (next2, next3) = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer)
    print(sess.run((next1, (next2, next3))))
    # iterator에 묶여 있어서 동시에 다음 element로 넘어간다.
    print(sess.run(next1)) 
    print(sess.run((next1, (next2, next3))))
    
    

### C. Reading input data

#### 1. Consuming NumPy arrays

In [ ]:
x = np.array([[1, 2, 3], [4, 5, 6]])
y = np.array([0, 1])

np.savez('./array', features=x, labels=y)
data = np.load('array.npz')

features = data['features']
labels = data['labels']

# The length of features and labels should be the same.
assert features.shape[0] == labels.shape[0]

feature_placeholder = tf.placeholder(features.dtype, features.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

dataset = tf.data.Dataset.from_tensor_slices((feature_placeholder, labels_placeholder))
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={feature_placeholder: features, labels_placeholder: labels})
    print(sess.run(next_element))
    print(sess.run(next_element))

#### 2. Consuming TFRecord data

#### 3. Consuming text data

### D. Preprocessing data with Dataset.map()

#### 1. Parsing tf.Example protocol buffer messages

#### 2. Decoding image data and resizing it

#### 3. Applying arbitrary Python logic with tf.py_func()

### E. Batching dataset elements

#### 1. Simple batching

In [ ]:
inc_dataset = tf.data.Dataset.range(100)
dec_dataset = tf.data.Dataset.range(0, -100, -1)
dataset = tf.data.Dataset.zip((inc_dataset, dec_dataset))
# basch 사이즈 지정

batched_dataset = dataset.batch(4)

iterator = batched_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    print(sess.run(next_element))  # ==> ([0, 1, 2,   3],   [ 0, -1,  -2,  -3])
    print(sess.run(next_element))  # ==> ([4, 5, 6,   7],   [-4, -5,  -6,  -7])
    print(sess.run(next_element))  # ==> ([8, 9, 10, 11],   [-8, -9, -10, -11])

#### 2. Batching tensors with padding

## Layers